# 本セクションの目次
1. Avroフォーマット
2. 前方互換と後方互換と完全互換
3. メッセージキューとAvroを連携してみよう
4. Avroファイルの読み書き
5. Avroで前方互換をやってみよう

# PySparkとは？
PySparkとは分散処理を実現するエンジン(フレームワークの様なもの)です。  
ビッグデータの世界では、一つのノードで処理を行うということはほとんどなく複数台のノードで一つの仕事を行い結果を出すということがしばしば行われます。  

その処理を比較的簡単に行なってくれるのがSparkであり、Pythonとの組み合わせて使うことでPySparkと呼ばれています。  
他にはScalaとの組み合わせやJavaとの組み合わせが可能ですが、ビッグデータ分析を行うという観点からPythonが多く利用されています。

単純に複数台並べて多くのリクエストを捌く様なWebシステムというわけではなくて、一つの大きな仕事をを処理するためのフレームワークと考えると良いと思います。

# 分散処理とは？

分散処理とは、複数の端末にまたがって処理を行うことです。  
スレッド処理とは異なり複数台にまたがっていることがポイントです。

例えば、「This is good」「That are very good」のgoodの数を数えるプログラムを考えるときの場合を考えてみます。

## スレッド処理の場合
一台のパソコン（端末）の中でスレッドを立ち上げ処理を行います。

端末1  
　 - スレッドの呼び出しもと  
    - スレッド1 「This is good」のgoodの数を数える  
    - スレッド2 「That are very good」のgoodの数を数える  
-> スレッド１、２がそれぞれgoodの数を数えてスレッドの呼び出し元にてそれぞれのスレッドの合計を取得し結果を表示する

## 分散処理の場合
複数台(今回は3台)で処理を行います。

コントローラーノード   
　- 端末1,端末2に指示を出して処理をさせる  
　- 端末1,端末2からの処理結果を受け取りユーザに返却する  

端末1
  - 「This is good」のgoodの数を数える

端末2
  - 「That are very good」のgoodの数を数える

分散処理の特徴は、スレッド処理としてノードを(実質)無限にスケールすることが可能な点です。  
仮にCPUやメモリが足りなければ端末3,4,,,,と増やしていくことで処理のボトルネックを解決することが可能になるという理論です。  
一方で、スレッド処理は一つの端末内で実行されるため、無限にスケールすることができません(CPU/メモリに限界がある)  

今回のコースは、データも小さく端末が一台(コントローラーノードと端末1が同居している状態)ですが実際の環境で構築する(された)環境を利用する際は複数台であることが一般的です。

# ノートブックとは？

ノートブックとはGUIで視覚的に分析を可能にした、まさにノートの様な分析環境です。  
プログラムやこの様にマークダウンを記載する場所を「セル」と呼びます。  
セルにプログラムやその説明を記載しながら状況を残していきます。

分析の結果をノートの様に一個ずつ残していけるからノートブックと呼ばれています。

このノートブック自体は他の人に共有することも可能で、実行したグラフの結果などをそのまま表示させてGitにコミットを行うことによって  
共有を行います。

今回はVSCodeのノートブック環境を利用していますが他にも

- jupyter notebook
- EMR notebook
- zeppeline

などなど、クラウド環境で提供されているものもあります。

# Spark(PySpark)がデータ操作で利用するもの

Sparkがデータを操作するときに利用するものは2つ(正確には3つ)あります。  

- SQL
- DataFrame
- RDD

今回のコースは、SQLとDataFrameをそれぞれ対比させながら紹介をおこなっていこうと思います。  
また、あまり出番はないのですが、RDDについても少し触れてみようと思います。

SQLとはデータを読み込みし、そのデータに対して仮想的なテーブルを作成しSQLを使ってデータ操作を行います。  
DataFrameとはデータを読み込みし、そのデータに対してプログラムチックにデータ操作を行います。  
RDDとはデータを読み込みし、そのデータに対してプログラムチックにデータ操作を行います(DataFrameよりより、低レベルのAPIを提供します)。

Sparkには2つの読み込みタイプが存在しています  
- スキーマオンリード(事前のテーブル定義がなくてもデータを読み込んで処理が可能)  
- スキーマオンライト(データを読み込むためには、事前のテーブル定義が必要)  

既存のテーブルからデータを読み取ることも可能ですしなくても問題ありません。

詳しくはセクション3でみていきましょう。

# どのセクションでも登場する前準備

どのセクションでも、データ処理を開始する前のおまじないがあります。

In [ ]:
# コンソールで設定したSparkとNoteBookを接続します(動かす前に毎度実行する必要があります)
import findspark
findspark.init("/home/pyspark/spark")

In [ ]:
#pysparkに必要なライブラリを読み込む
from pyspark import SparkConf
from pyspark import SparkContext
from pyspark.sql import SparkSession

#spark sessionの作成
# spark.ui.enabled trueとするとSparkのGUI画面を確認することができます
# spark.eventLog.enabled true　とすると　GUIで実行ログを確認することができます
# GUIなどの確認は最後のセクションで説明を行います。
spark = SparkSession.builder \
    .appName("chapter1") \
    .config("hive.exec.dynamic.partition", "true") \
    .config("hive.exec.dynamic.partition.mode", "nonstrict") \
    .config("spark.sql.session.timeZone", "JST") \
    .config("spark.ui.enabled","true") \
    .config("spark.eventLog.enabled","true") \
    .config("spark.jars.packages", "org.apache.spark:spark-streaming_2.13:3.2.0,org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.0,org.apache.spark:spark-avro_2.12:3.2.0") \
    .enableHiveSupport() \
    .getOrCreate()

# Spark SessionはJavaでいうインスタンスの様なもので、spark(=分散処理しますよ)というマークだと考えればOK
# それ以外はconfigで細かな設定が可能
# パッケージを複数渡したい時は「,」で繋いで渡します。
# Sparkのバージョンにしっかりと合わせます(今回はSparkのバージョンが3.2を使っています。)。

In [ ]:
spark.stop()